# Implementing Dissipative Particle Dynamics in Molly

The aim of this workshop is to go through the process of implementing a numerical method from scratch using the software package [Molly](https://github.com/JuliaMolSim/Molly.jl), which is a Julia package for molecular simulation. Along the way, we will get familiar with some of Molly's many functionalities designed to make molecular simulation easy, as well as some of the tricks that go into implementing new ones.

To run this notebook, you will be needing a working installation of [Julia](https://julialang.org/) (we suggest the current stable release), as well as [IJulia](https://juliapackages.com/p/ijulia).

We setup the notebook to use Molly and GLMakie for visualization purposes.


In [21]:
import Pkg

Pkg.add("Molly")
Pkg.add("GLMakie") # for visualization
Pkg.add("Plots")

using Molly, GLMakie, Random, LinearAlgebra, Plots

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


In [15]:

struct DPDInteraction{S,T,U} <: PairwiseInteraction
    nl_only :: Bool # a Boolean parameter which forces the simulator to use the neighbor list
    cutoff_radius :: S # cutoff radius
    coefficients :: T # the parameters of the interaction, we will assume that it is a function f(i,j)->a_ij
    
    sq_cutoff_radius :: U
    inv_cutoff_radius :: V

    force_units :: F
    energy_units :: E 
end

function DPDInteraction(nl_only, cutoff_radius,coefficients ; force_units = NoUnits, energy_units = NoUnits)
    #constructor to spare the user from computing the squared cutoff radius automatically
    
    sq_cutoff_radius = cutoff_radius^2
    inv_cutoff_radius = inv(cutoff_radius)
    
    S=typeof(cutoff_radius)
    T=typeof(coefficients)
    
    U=typeof(sq_cutoff_radius)
    V=typeof(inv_cutoff_radius)
    
    F = typeof(force_units)
    E = typeof(energy_units)
    
    return DPDInteraction{S,T,U,V,F,E}(nl_only,cutoff_radius,sq_cutoff_radius,coefficients,force_units,energy_units)
end

LoadError: invalid redefinition of constant DPDInteraction

In [8]:
function Molly.force(inter::DPDInteraction{S,T}, dr, qi, qj, atom_i, atom_j, boundary) where {S,T}    
    r2=sum(abs2,dr)
    (r2 > inter.sq_cutoff_radius) && return zero(dr) # set force to zero beyond the cutoff radius
    
    i = atom_i.index
    j = atom_j.index 
    a_ij = inter.coefficients(i,j)
    
    return a_ij*(inv(sqrt(r2)) - inter.inv_cutoff_radius)*dr
    
end

In [26]:
struct DPDSplitting{S,T,U,V,W}
    weights::S # assume weights is a function r -> wR(r)
    friction::T
    β::U
    splitting::V
    dt::W
    
    remove_CM_motion::Bool
end

function DPDSplitting(;weights, friction, temperature, k=1.0, splitting ,dt, remove_CM_motion = True)
    S = typeof(weights)
    T = typeof(friction)
    β = inv(k*temperature),w
    U = typeof(β)
    V = typeof(splitting)
    W=typeof(dt)
    
    return DPDSplitting{S,T,U,V}(weights,friction,β,splitting, remove_CM_motion)
end

LoadError: invalid redefinition of constant DPDSplitting

In the next step, we define the skeleton for our integrator. This is done by implementing a method for `Molly.simulate` for a simulator of type `DPDSplitting`. This function sets up the simulation, and is in charge of running the iterations. It is almost line for line the same as the `Molly.simulate!` method for the `LangevinSplitting` type of the simulator argument, so maybe the correct way to implement this for "production" purposes would be to define a common type for all splitting integrators (e.g. Langevin, DPD, Norton...) and have each of these simulator types define a method for the corresponding A,B and O steps.

In [28]:
function Molly.simulate!(sys,simulator::DPDSplitting,n_steps; n_threads=Threads.nthreads(), rng = Random.GLOBAL_RNG)
    M_inv = inv.(masses(sys))


    sys.coords = wrap_coords.(sys.coords, (sys.boundary,))
    sim.remove_CM_motion && remove_CM_motion!(sys)
    neighbors = find_neighbors(sys, sys.neighbor_finder; n_threads=n_threads)
    run_loggers!(sys, neighbors, 0; n_threads=n_threads)
    accels_t = accelerations(sys, neighbors; n_threads=n_threads)

    effective_dts = [sim.dt / count(c, sim.splitting) for c in sim.splitting]

    # Determine the need to recompute accelerations before B steps
    forces_known = !occursin(r"^.*B[^B]*A[^B]*$", sim.splitting)

    force_computation_steps = map(collect(sim.splitting)) do op
        if op == 'O'
            return false
        elseif op == 'A'
            forces_known = false
            return false
        elseif op == 'B'
            if forces_known
                return false
            else
                forces_known = true
                return true
            end
        end
    end

    step_arg_pairs = map(enumerate(sim.splitting)) do (j, op)
        if op == 'A'
            return (Molly.A_step!, (sys, effective_dts[j]))
        elseif op == 'B'
            return (Molly.B_step!, (sys, effective_dts[j], accels_t, force_computation_steps[j], n_threads))
        elseif op == 'O'
            return (O_step_DPD!, (sys, effective_dts[j], rng, sim.temperature,sim.weights, n_threads))
        end
    end
    
     for step_n in 1:n_steps
        old_coords = copy(sys.coords)
        for (step!, args) in step_arg_pairs
            step!(args..., neighbors)
        end
        
        apply_constraints!(sys, old_coords, sim.dt)
        sys.coords = wrap_coords.(sys.coords, (sys.boundary,))
        sim.remove_CM_motion && remove_CM_motion!(sys)
        run_loggers!(sys, neighbors, step_n)
        
        if step_n != n_steps
            neighbors = find_neighbors(sys, sys.neighbor_finder, neighbors, step_n;
                                        n_threads=n_threads)
        end
    end
    return sys
    
end



The A and B steps are the same as for the standard Langevin method, they have already been implemented in Molly for the purpose of the `LangevinSplitting` integrator, so we only need to implement the O step.

In [27]:
function O_step_DPD!(sys,dt,rng,sim.temperature,sim.weights,neighbors)
    for (i,j,_) = neighbors.list
        dr = vector(sys.coords[i],sys.coords[j],sys.boundary)
        
        r = sqrt(sum(abs2,dr))
        dv = sys.velocities[j]-sys.velocities[i]
        vij=dot(dr,dv)/r #relative velocity
        
        mi=sys.atoms[i].mass
        mj=sys.atoms[j].mass
        
        mij=(mi*mj)/(mi+mj) #relative mass
        
        w=sim.weights(r)
        
    end
    
end

A_step_DPD! (generic function with 1 method)